In [1]:
# script to load the diffrent node Embeddings and save them to feather file 
# needs triple_array.txt and nodes.txt file with all nodes from the Graph! (len = 549872)
# at first, create list of nodes (name them according to graph)

# TODO: add load triple from path
import pykeen
from pykeen.pipeline import pipeline
import pandas as pd
import numpy as np
from pykeen.constants import PYKEEN_CHECKPOINTS
from pykeen.triples import TriplesFactory
import torch


Task was destroyed but it is pending!
task: <Task pending name='Task-2' coro=<Kernel.poll_control_queue() running at /sc-projects/sc-proj-ukb-cvd/environments/gnn/lib/python3.9/site-packages/ipykernel/kernelbase.py:227> wait_for=<Future finished result=[<zmq.sugar.fr...x7f3ed99800f0>, <zmq.sugar.fr...x7f3ec4736d50>, <zmq.sugar.fr...x7f3ec4736e00>, <zmq.sugar.fr...x7f3ec4736eb0>, <zmq.sugar.fr...x7f3ec4736f60>, <zmq.sugar.fr...x7f3ec4739040>, ...]> cb=[_chain_future.<locals>._call_set_state() at /sc-projects/sc-proj-ukb-cvd/environments/gnn/lib/python3.9/asyncio/futures.py:391]>


In [ ]:
# 2 load the graph data and preprocess
# from https://github.com/nebw/ehrgraphs/blob/master/ehrgraphs/data/data.py#L82-L117
import networkx as nx
import pathlib
def preprocess_graph_heterogeneous(graph: nx.Graph):
    edge_types = []
    for u, v, data in graph.edges.data():
        edge_types.append(data["edge_type"])

    edge_codes, edge_types = pd.factorize(edge_types)

    node_types = []
    for n, data in graph.nodes.data():
        node_types.append(data["node_type"])

    node_codes, node_types = pd.factorize(node_types)

    preprocessed_graph = nx.DiGraph()
    preprocessed_graph.add_nodes_from(graph.nodes())

    preprocessed_graph.node_codes = node_codes
    preprocessed_graph.node_types = node_types
        
    # drop shortcut edges
    exclude_codes = []
    exclude_codes.append(edge_codes[list(edge_types).index("Subsumes")])
    exclude_codes.append(edge_codes[list(edge_types).index("Is a")])

    for (u, v, w), c in zip(graph.edges.data("edge_weight"), edge_codes):
        assert w is not None

        # drop shortcut edges
        if c in exclude_codes and w < 1.0:
            continue

        preprocessed_graph.add_edge(u, v, edge_weight=w, edge_code=c)

    preprocessed_graph.edge_types = edge_types
    
    

    return preprocessed_graph

#2 load graph data 
#loading the full graph
base_path = pathlib.Path(
    "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning")
G = nx.readwrite.gpickle.read_gpickle('/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning/graph_full_211122.p')
#2 preprocess now 
PG = preprocess_graph_heterogeneous(G)
#2 create Triple Array
tripleList=[]
nodes=[]
for u,v,data in PG.edges.data():
    l=[]
    l.append(u)
    nodes.append(u)
    l.append(data['edge_code'])
    l.append(v)
    nodes.append(v)
    tripleList.append(l)

#needs triples as ndarray - shape (n,3), dtype:str 
tripleArray=np.array(tripleList, dtype=str)
print(len(tripleArray))
del G
del PG

In [ ]:
# save nodelist and tripleArray as txt file 
'''with open("node_list.txt", 'w') as f:   
    for n in nodes:
        f.write(str(n) +'\n')

with open('triple_array.txt', 'w') as f:
    for liste in tripleArray:
        row =liste[0] + ' ' + liste[1] + ' ' + liste[2]
        f.write(str(row) + '\n')

# load node_list.txt 
nodes2 = []
with open('node_list.txt', 'r') as f:
    for line in f:
        nodes.append(str(line.strip()))

tripleArray2=[]
with open('triple_array.txt', 'r') as f:
    for line in f:
        tripleArray2.append(l)
        
'''
# save nodelist and triples array as pd.feather

df_nodes = pd.DataFrame(nodes, columns = ['nodes'])
df_triple = pd.DataFrame(tripleArray, columns = ['n1', 'relation', 'n2'])
print(df_nodes.head())
df_triple.head()

df_nodes.to_feather('node_list.feather')
df_triple.to_feather('triple_list.feather')

In [4]:
# 111111111111111111111111111111111111111111111111

#! 1. pay attention to the embedding_dim variable and addapt it if necessary

# function to load the embedding 

def load_embedding(checkpoint_path, model, emb_dim, tripleArray,inverse, cuda=False):
    if cuda is False:
        checkpoint = torch.load(PYKEEN_CHECKPOINTS.joinpath(checkpoint_path), map_location={'cuda:0':'cpu'}) 
        # checkpoint['entity_to_id_dict']) 
    else:
        checkpoint = torch.load(PYKEEN_CHECKPOINTS.joinpath(checkpoint_path))
    
    
    # checkpoint contains model and entity_to_id, realtion_to_id mapping that was used
    # load these into Pykeen:
    train_tf = TriplesFactory.from_labeled_triples(
        triples = tripleArray, 
        create_inverse_triples=inverse, # default was True due 18.03.22
        entity_to_id=checkpoint['entity_to_id_dict'],
        relation_to_id=checkpoint['relation_to_id_dict'],
    )
    
    # load the model and pass the train triple factory to the model
    used_model = model(
        triples_factory=train_tf,
        embedding_dim=emb_dim, # 128 if RotatE or ComplEx (b.c. of complex integers)       # check necessary
        random_seed=420
    ) # 128 for complex embeddings
    
    
    # get the embeddings
    # class RepresentationModule
    entity_RepModel = used_model.entity_representations[0] # check for more representations
    try:
        print(used_model.entity_representations[1])
    except IndexError:
        print('Index Error, no more entity_reps ')
    
    #TODO assert for entity_representations[1]
    
    # accses over checkpoint: 
    checkp_entities_val = list(checkpoint['entity_to_id_dict'].values())
    embeddings = entity_RepModel(torch.tensor(checkp_entities_val, dtype=torch.long)) # error with NopePiece at dtype=torch.int()# cast list elements into tensors
    #??embedding_dict = dict(zip(checkpoint['entity_to_id_dict'].keys(), embeddings.detach().numpy() )) #  Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
    df = pd.DataFrame(dict(nodes= checkpoint['entity_to_id_dict'].keys(), embeddings= list(embeddings.detach().numpy()) ))
    del model
    return df


# takes necessary arguments and passes it to function. checks all necessary variables
def helper(checkpoint_name, Model,emb_dim, emb_dict_name, inverse):
    print(Model)
    check = input("imported the right Model? inverse Triple used? /yes")
    if check != "yes": return 1
    #load the triplesArray
    triples = pd.read_feather('triple_list.feather').to_numpy()
    checkp_path = checkpoint_name + ".pt" #  "'/home/tilingl/.data/pykeen/checkpoints/"
    df = load_embedding(checkp_path, Model, emb_dim, triples,inverse)
    print(df)
    emb_dic_path ="embeddings/Embedding_dict_" + emb_dict_name + ".feather"
    df.to_feather(emb_dic_path)
    
    return 0


In [5]:
# 22222222222222222222222222222222222222222222

# save embeddings/ embedding_dict as feather

#! 1. modify the path according to the right Embedding checkpoint
#! 2. Import the corresponding model from pykeen.models
#! 3. Change the model variable to the relating model
#! 4. adapt the name of the new Embedding_dict according to the checkpoint/model and Version of the embedding
#! 5. check the embedding_dim variable in the load_embedding function


from pykeen.models import RotatE # ! 128 if RotatE or ComplEx (b.c. of complex integers)
#helper(checkpoint_name, Model,emb_dim, emb_dict_name, inverse_triple?)
helper("RotatE_2_256_t2",RotatE ,128, "RotatE_2_256_t2_v2",False)

<class 'pykeen.models.unimodal.rotate.RotatE'>


imported the right Model? inverse Triple used? /yes yes


No cuda devices were available. The model runs on CPU


Index Error, no more entity_reps 
                  nodes                                         embeddings
0          OMOP_1000560  [0.26667762, -0.28079098, 0.22935323, -0.37219...
1          OMOP_1000577  [0.24906819, -0.3066625, -0.186661, -0.2085861...
2          OMOP_1000579  [-0.07856253, 0.14064436, 0.34394872, 0.140200...
3          OMOP_1000599  [0.27964348, -0.1326068, 0.3160723, 0.1552092,...
4          OMOP_1000600  [0.17873678, 0.21795434, -0.121780336, -0.3138...
...                 ...                                                ...
511286  OT_R-HSA-983695  [0.16453658, 0.17231901, 0.331063, 0.036961824...
511287  OT_R-HSA-983705  [0.24189198, 0.060960844, 0.060015544, -0.3292...
511288  OT_R-HSA-983712  [-0.120542586, 0.28354615, -0.2839713, 0.03951...
511289  OT_R-HSA-991365  [0.34917247, -0.17682476, 0.11770615, -0.02588...
511290  OT_R-HSA-997272  [-0.29147065, -0.071174115, 0.1486845, 0.14663...

[511291 rows x 2 columns]


0

In [44]:
# check out MuRE second entity_representation and wether it is important or not 
#MuRe_model = helper("MuRE_slcwa_t1",MuRE ,256, "MuRE_slcwa_t1_v2")
checkpoint = torch.load(PYKEEN_CHECKPOINTS.joinpath("MuRE_slcwa_t1.pt"), map_location={'cuda:0':'cpu'}) 
#triples = pd.read_feather('triple_list.feather').to_numpy()
#train = TriplesFactory.from_labeled_triples(
#    triples=triples,
#    entity_to_id=checkpoint['entity_to_id_dict'],
#    relation_to_id=checkpoint['relation_to_id_dict'],

#)
checkpoint[
#now load the model and pass the train triples factory to the model
my_model = MuRE(
    triples_factory=train, 
    embedding_dim=256,
    random_seed=600)

my_model.load_state_dict(checkpoint['model_state_dict'])

print(my_model.entity_representations[0], my_model.entity_representations[1])



# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
# in comparison to old method: check instant avaiability of variables


No cuda devices were available. The model runs on CPU


RuntimeError: Error(s) in loading state_dict for MuRE:
	size mismatch for relation_representations.0._embeddings.weight: copying a param with shape torch.Size([326, 256]) from checkpoint, the shape in current model is torch.Size([163, 256]).
	size mismatch for relation_representations.1._embeddings.weight: copying a param with shape torch.Size([326, 256]) from checkpoint, the shape in current model is torch.Size([163, 256]).

In [53]:
print(checkpoint.keys(), len(checkpoint['model_state_dict']))
file = checkpoint['model_state_dict']
print('\n',file.keys())

dict_keys(['epoch', 'loss', 'model_state_dict', 'optimizer_state_dict', 'lr_scheduler_state_dict', 'checksum', 'random_seed', 'stopper_dict', 'random_state', 'np_random_state', 'torch_random_state', 'torch_cuda_random_state', 'best_epoch_model_checkpoint', 'relation_to_id_dict', 'entity_to_id_dict']) 5

 odict_keys(['entity_representations.0._embeddings.weight', 'entity_representations.1._embeddings.weight', 'entity_representations.2._embeddings.weight', 'relation_representations.0._embeddings.weight', 'relation_representations.1._embeddings.weight'])


In [ ]:
triples = pd.read_feather('triple_list.feather').to_numpy()

#returns dataframe with dict of nodes:embedding
from pykeen.models import ComplEx # !
df = load_embedding('/home/tilingl/.data/pykeen/checkpoints/ComplEx_t1.pt', ComplEx, triples)
print(df)
df.to_feather('embeddings/Embedding_dict_ComplEx_2_t1_v2.feather')

# notitz: MuRE hat noch eine entity representation

# Explaining variable baselines
# model.pt -> baseline
# model_2.pt -> fixed some errors in baseline
# model_t1.pt -> first hyperparameter tuning 
# model_256/128.pt -> embedding_dim specification for the used model
# model_v2.pt -> due to a missmatch of embedding model and checkpoint model this is version 2 with matched models. All embeddings after february where missmatched with RotatE

# beginning at 27.03.22 new script and new approach to achieve embeddins!
# emb_runNAME called rus in wandb
# emb_runNAME_tx